# CryptoCurrency Notebook

### Install coinmarketcap
This is a python wrapper to the cryptocurrency API.  More often than not, I try to leverage python wrappers over direct API calls, because many of them have already worked out the kinks, added error handling, etc.

In [3]:
!pip install coinmarketcap

  Running setup.py bdist_wheel for coinmarketcap ... done
  Stored in directory: /content/.cache/pip/wheels/5c/73/ec/47f4d3160b8d215cc223937a3886eccfc690cd3dbb5152ab42
Successfully built coinmarketcap


In [4]:
!pip install google.cloud.logging

  Using cached https://files.pythonhosted.org/packages/20/9c/f0c9011b334f250977dd52bae260352587f86a9c6b03e770d6954d9c8405/google_cloud_logging-1.8.0-py2.py3-none-any.whl


### Install pandas
For small amounts of data, I like to leverage pandas for data manipulation.

In [5]:
!pip install pandas
!pip install google.cloud.bigquery

    100% |████████████████████████████████| 92kB 3.4MB/s ta 0:00:011
google-cloud-dataflow 2.0.0 has requirement google-cloud-bigquery<0.24.0,>=0.23.0, but you'll have google-cloud-bigquery 1.6.0 which is incompatible.
google-cloud-dataflow 2.0.0 has requirement httplib2<0.10,>=0.8, but you'll have httplib2 0.11.3 which is incompatible.
google-cloud-dataflow 2.0.0 has requirement protobuf==3.2.0, but you'll have protobuf 3.5.2 which is incompatible.


### Import csv, Market, Logging, and pandas
Import the necessary libraries and setup the API connection.

In [1]:
import csv
from coinmarketcap import Market
import pandas as pd

coinmarketcap = Market()

import google.cloud.logging
client = google.cloud.logging.Client()

import logging

handler = client.get_default_handler()
cloud_logger = logging.getLogger('cloudLogger')
cloud_logger.setLevel(logging.INFO)
client.setup_logging(log_level=logging.INFO)

### Max Loops
By querying the listings, I get a number of records available in the ticker.  By doing this, I'll be able to control my loop later, so I get ticker information in chunks of 100 records (the max for that API).

In [2]:
try:
    listings_return = coinmarketcap.listings()
    listings_df = pd.DataFrame.from_dict(listings_return['data'])
    max_loop = listings_df['id'].count()

    logging.info('Total Listings: ' + str(max_loop))
  
except Exception as e:
    logging.critical("FAILED: " + str(e))
    raise ValueError(str(e))

Total Listings: 2112


### Run the Loop
Loop through the API to get every record.  This process needs to transpose the data correctly into columns and then append each loop to a persistent "full" dataframe.

In [3]:
try:
    j = 1
    while j < max_loop:
        ticker_return = coinmarketcap.ticker(start=j, limit=100)
        ticker_df = pd.DataFrame.from_dict(ticker_return['data'])
        if ticker_df.shape[0] != 0:
            ticker_trans = ticker_df.transpose()
            if j == 1:
                ticker_full = ticker_trans
            else:
                ticker_full = ticker_full.append(ticker_trans)
            j += 100
        logging.info('Listing: ' + str(j))
    
    logging.info('Dataframe Built')

except Exception as e:
    logging.critical("FAILED: " + str(e))
    raise ValueError(str(e))
    
ticker_full

Listing: 101
Listing: 201
Listing: 301
Listing: 401
Listing: 501
Listing: 601
Listing: 701
Listing: 801
Listing: 901
Listing: 1001
Listing: 1101
Listing: 1201
Listing: 1301
Listing: 1401
Listing: 1501
Listing: 1601
Listing: 1701
Listing: 1801
Listing: 1901
Listing: 2001
Listing: 2101
Listing: 2201
Dataframe Built


,circulating_supply,id,last_updated,max_supply,name,quotes,rank,symbol,total_supply,website_slug
1,1.73359e+07,1,1540223382,2.1e+07,Bitcoin,"{u'USD': {u'market_cap': 1.12632577238e+11, u'...",1,BTC,1.73359e+07,bitcoin
1027,1.02731e+08,1027,1540223383,None,Ethereum,"{u'USD': {u'market_cap': 21037638491.0, u'perc...",2,ETH,1.02731e+08,ethereum
1042,3.75211e+10,1042,1540223375,None,Siacoin,"{u'USD': {u'market_cap': 255010373.0, u'percen...",37,SC,3.75211e+10,siacoin
109,1.09557e+10,109,1540223410,2.1e+10,DigiByte,"{u'USD': {u'market_cap': 248568875.0, u'percen...",39,DGB,1.09557e+10,digibyte
1104,1.1e+07,1104,1540223373,None,Augur,"{u'USD': {u'market_cap': 144999567.0, u'percen...",51,REP,1.1e+07,augur
1168,8.68447e+06,1168,1540223373,2.1e+07,Decred,"{u'USD': {u'market_cap': 335408035.0, u'percen...",28,DCR,8.68447e+06,decred
1169,5.67812e+07,1169,1540223371,None,PIVX,"{u'USD': {u'market_cap': 74006395.0, u'percent...",88,PIVX,5.67812e+07,pivx
118,2.88087e+10,118,1540223412,None,ReddCoin,"{u'USD': {u'market_cap': 86700498.0, u'percent...",70,RDD,2.88087e+10,reddcoin
1214,1.11431e+08,1214,1540223377,None,Lisk,"{u'USD': {u'market_cap': 329900043.0, u'percen...",29,LSK,1.26668e+08,lisk
1229,2e+06,1229,1540223371,None,DigixDAO,"{u'USD': {u'market_cap': 87278096.0, u'percent...",69,DGD,2e+06,digixdao


### Parse Nested Data
Within the resulting dataframe is another nested dictionary of data "quotes".  This step parses that data would and creates a new column for each value that I want to store.

In [4]:
try:
    ticker_full['market_cap'] = ticker_full.apply(lambda row: row.quotes['USD'][u'market_cap'], axis=1)
    ticker_full['percent_change_7d'] = ticker_full.apply(lambda row: row.quotes['USD'][u'percent_change_7d'], axis=1)
    ticker_full['price'] = ticker_full.apply(lambda row: row.quotes['USD'][u'price'], axis=1)
    ticker_full['percent_change_1h'] = ticker_full.apply(lambda row: row.quotes['USD'][u'percent_change_1h'], axis=1)
    ticker_full['volume_24h'] = ticker_full.apply(lambda row: row.quotes['USD'][u'volume_24h'], axis=1)
    ticker_full['percent_change_24h'] = ticker_full.apply(lambda row: row.quotes['USD'][u'percent_change_24h'], axis=1)
    
    logging.info('Dataframe Modified')
    
except Exception as e:
    logging.critical("FAILED: " + str(e))
    raise ValueError(str(e))    

ticker_full

Dataframe Modified


,circulating_supply,id,last_updated,max_supply,name,quotes,rank,symbol,total_supply,website_slug,market_cap,percent_change_7d,price,percent_change_1h,volume_24h,percent_change_24h
1,1.73359e+07,1,1540223382,2.1e+07,Bitcoin,"{u'USD': {u'market_cap': 1.12632577238e+11, u'...",1,BTC,1.73359e+07,bitcoin,1.126326e+11,-2.06,6497.061867,-0.04,3.623062e+09,-0.58
1027,1.02731e+08,1027,1540223383,None,Ethereum,"{u'USD': {u'market_cap': 21037638491.0, u'perc...",2,ETH,1.02731e+08,ethereum,2.103764e+10,-2.23,204.784708,0.01,1.367291e+09,-1.06
1042,3.75211e+10,1042,1540223375,None,Siacoin,"{u'USD': {u'market_cap': 255010373.0, u'percen...",37,SC,3.75211e+10,siacoin,2.550104e+08,4.83,0.006796,0.16,4.109611e+06,-2.35
109,1.09557e+10,109,1540223410,2.1e+10,DigiByte,"{u'USD': {u'market_cap': 248568875.0, u'percen...",39,DGB,1.09557e+10,digibyte,2.485689e+08,-5.93,0.022689,-1.05,2.299384e+06,-3.71
1104,1.1e+07,1104,1540223373,None,Augur,"{u'USD': {u'market_cap': 144999567.0, u'percen...",51,REP,1.1e+07,augur,1.449996e+08,6.95,13.181779,1.08,1.665098e+06,0.10
1168,8.68447e+06,1168,1540223373,2.1e+07,Decred,"{u'USD': {u'market_cap': 335408035.0, u'percen...",28,DCR,8.68447e+06,decred,3.354080e+08,-0.79,38.621579,-0.20,5.831934e+05,0.98
1169,5.67812e+07,1169,1540223371,None,PIVX,"{u'USD': {u'market_cap': 74006395.0, u'percent...",88,PIVX,5.67812e+07,pivx,7.400640e+07,15.87,1.303362,0.63,6.044741e+05,0.32
118,2.88087e+10,118,1540223412,None,ReddCoin,"{u'USD': {u'market_cap': 86700498.0, u'percent...",70,RDD,2.88087e+10,reddcoin,8.670050e+07,-9.20,0.003010,-0.23,6.508855e+05,-4.45
1214,1.11431e+08,1214,1540223377,None,Lisk,"{u'USD': {u'market_cap': 329900043.0, u'percen...",29,LSK,1.26668e+08,lisk,3.299000e+08,2.57,2.960580,0.14,6.456297e+06,1.53
1229,2e+06,1229,1540223371,None,DigixDAO,"{u'USD': {u'market_cap': 87278096.0, u'percent...",69,DGD,2e+06,digixdao,8.727810e+07,-0.21,43.639048,-0.54,4.339191e+05,-3.69


### Drop to CSV
Using the pandas function to create a CSV from the dataframe, I create a file from the df without the nested quotes column, which has now been split into their own columns.

In [5]:
try:
    ticker_full.drop(columns=['quotes']).to_csv('cryptocurrency_details.csv',index=False, quoting=csv.QUOTE_NONNUMERIC)
    logging.info('CSV File Generated')
    
except Exception as e:
    logging.critical("FAILED: " + str(e))
    raise ValueError(str(e))

CSV File Generated


### gsutil - Copy File to GCS
__Note:__ The bucket was created manually within the GUI when I created the project.  This could have been done through the command line, as well, but was easier in the interface since I was already there.

In [6]:
!gsutil cp cryptocurrency_details.csv gs://hokie-crazy-public/cryptocurrency/cryptocurrent_details.csv



Updates are available for some Cloud SDK components.  To install them,
please run:
  $ gcloud components update

Copying file://cryptocurrency_details.csv [Content-Type=text/csv]...
/ [1 files][267.7 KiB/267.7 KiB]                                                
Operation completed over 1 objects/267.7 KiB.                                    


### gsutil - Make the bucket public
__Note:__ This command adds all users as a read-only user to the bucket where the file above is located.

In [20]:
!gsutil iam ch allUsers:objectViewer gs://hokie-crazy-public

### Load the BigQuery Library
__Note:__ This command allows me to use %%bigquery magics to execute queries.  Might not be necessary to load this in DataLab, but to be safe...

In [7]:
%load_ext google.cloud.bigquery

### How many coins have a USD price greater than $8,000?

In [14]:
%%bigquery
SELECT COUNT(id)
FROM `hokie-crazy.cryptocurrency.ticker`
WHERE price > 8000

#3

,f0_
0,3


### What is the total market cap of the top 100 cryptocurrencies (in USD)?

In [15]:
%%bigquery
SELECT SUM(market_cap)
FROM (SELECT market_cap, ROW_NUMBER() OVER (ORDER BY market_cap DESC) as row_num FROM `hokie-crazy.cryptocurrency.ticker`) a
WHERE a.row_num <= 100

#2.027954e+11

,f0_
0,2.027954e+11


### Which coins have an available supply less than $5M?

In [16]:
%%bigquery
SELECT name
FROM `hokie-crazy.cryptocurrency.ticker`
WHERE circulating_supply * price < 5000000

,name
0,CryptoCarbon
1,MarteXcoin
2,Bitcloud
3,Ultimate Secure Cash
4,BriaCoin
5,Golfcoin
6,Granite
7,Gold Pressed Latinum
8,HoboNickels
9,CacheCoin


### Which 5 coins have seen the greatest percentage growth in the last week?

In [17]:
%%bigquery
SELECT a.name
FROM (SELECT name, ROW_NUMBER() OVER (ORDER BY percent_change_7d DESC) as row_num FROM `hokie-crazy.cryptocurrency.ticker`) a
WHERE a.row_num <= 5

,name
0,BBSCoin
1,Cobrabytes
2,FREE Coin
3,PitisCoin
4,TRONCLASSIC


### How many ticker symbols contain the letter "X"?

In [19]:
%%bigquery
SELECT COUNT(id)
FROM `hokie-crazy.cryptocurrency.ticker`
WHERE symbol like '%X%'

#271

,f0_
0,271
